### - Function Calling (to connect models to external data and public APIs)


> **Function** calling enables developers to connect language models to external data and systems. You can define a set of functions as tools that the model has access to, and it can use them when appropriate based on the conversation history. You can then execute those functions on the application side, and provide results back to the model.

In [ ]:
pip install openai==1.56.1 httpx==0.27.2 request

- [Get an API key](https://platform.openai.com/account/api-keys)
- OPENAI_API_KEY="your api key"

In [ ]:
from openai import OpenAI
import requests
import json
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')

client = OpenAI(api_key=api_key)
WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')

### [OpenWeatherMap API](https://openweathermap.org/api)
[1] - create new account\
[2] - get an API Key to send requests and collect weather data

utils functions
1. unit of temperature 
2. get coordonates

In [ ]:

def kelvin_to_celsius(kelvin):
    return json.dumps(round(kelvin - 273.15))


def kelvin_to_fahrenheit(kelvin):
    return round((kelvin - 273.15) * 9 / 5 + 32, 2)


def geo_code(location):
    loc = location.split(",")[0]
    url = (
        f"http://api.openweathermap.org/geo/1.0/direct?q={loc}&appid={WEATHER_API_KEY}"
    )

    try:
        response = requests.get(url)
        response.raise_for_status()
        coordinates = response.json()
        lat = coordinates[0].get("lat")
        lon = coordinates[0].get("lon")
        return lat, lon

    except requests.HTTPError as err:
        print(f"HTTP error occurred: {err}")
        return None, None


In [ ]:

def get_current_weather(location, unit="celsius"):
    lat, lon = geo_code(location)

    if lat is None or lon is None:
        print("Failed to get location coordinates.")
        return

    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={WEATHER_API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        weather_data = response.json()
        current_temp = weather_data["main"]["temp"]
        description = weather_data["weather"][0]["description"]

        weather_info = {
            "location": location,
            "temperature": kelvin_to_celsius(current_temp)
            if unit == "celsius"
            else kelvin_to_fahrenheit(current_temp),
            "unit": unit,
            "forecast": description,
        }

        # make sure to convert to stringified json object
        return json.dumps(weather_info)

    except requests.HTTPError as err:
        print(f"HTTP error occurred: {err}")
        return


### Function Calling

In [ ]:
# Constants
MODEL_ENGINE = "gpt-4o-mini"
messages = [{"role": "system", "content": "You are a helpful assistant"}]
user_input = "What's the weather in San Francisco?"
available_functions = {
    "get_current_weather": get_current_weather,
}  # only one function in this example, but you can have multiple


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location", "unit"],
            },
        },
    }
]


def generate_response():
    # Step 1: send the conversation and available functions to GPT
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": user_input}],
    tools=tools,
    )
    messages.append(completion.choices[0].message)  # extend conversation with assistant's reply

    # Step 2: check if GPT wanted to call a function and generate an extended response
    tool_calls = completion.choices[0].message.tool_calls
    print(completion.choices[0].message.tool_calls)

    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            # Step 3: call the function
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )

            print(function_response)

            # Step 4: send the info on the function call and function response to GPT
            # extend conversation with assistant's reply
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
            # Step 4: send the info on the function call and function response to GPT

        # extend conversation with assistant's reply
        second_response = client.chat.completions.create(
          model="gpt-3.5-turbo-1106",
          messages=messages,
        )  # get a new response from the model where it can see the function response
        print("Bot: " + second_response.choices[0].message.content)



In [ ]:
message_response = generate_response()
print(message_response)